In [1]:
import pulp
from pulp import *

In [2]:
import pandas as pd

## Create Data

In [3]:
demand = {'Demand': {'A': 145.4, 
                     'B': 84.1, 
                     'C': 156.4, 
                     'D': 1676.8, 
                     'E': 2719.6}}

var_cost = {'A': {'A': 8, 'B': 14, 'C': 23, 'D': 21, 'E': 12}, 
            'B': {'A': 14, 'B': 6, 'C': 13, 'D': 14, 'E': 13}, 
            'C': {'A': 21, 'B': 13, 'C': 8, 'D': 9, 'E': 17}, 
            'D': {'A': 21, 'B': 14, 'C': 10, 'D': 3, 'E': 20}, 
            'E': {'A': 12, 'B': 13, 'C': 22, 'D': 20, 'E': 6}}

fix_cost = {'High_C': {'A': 4730, 
                       'B': 7270, 
                       'C': 3080, 
                       'D': 9100, 
                       'E': 9500},
            
            'Low_C': {'A': 3230, 
                      'B': 4980, 
                      'C': 2110, 
                      'D': 6230, 
                      'E': 6500}}

capacity = {'High_C': {'A': 1500, 
                       'B': 1500, 
                       'C': 1500, 
                       'D': 1500, 
                       'E': 1500},
            
            'Low_C': {'A': 500, 
                      'B': 500, 
                      'C': 500, 
                      'D': 500, 
                      'E': 500}}

In [4]:
df_demand = pd.DataFrame(demand)
df_var_cost = pd.DataFrame(var_cost)
df_fix_cost = pd.DataFrame(fix_cost)
df_capacity = pd.DataFrame(capacity)

## Initialize Model

In [5]:
model = LpProblem('Facility_Location_Problem', LpMinimize)

## Define Decision Variables

In [6]:
location = ['A', 'B', 'C', 'D', 'E']
capacity = ['Low_C', 'High_C']

In [7]:
X = LpVariable.dicts('production_', [(i, j) for i in location for j in location], 
                      lowBound=0, upBound=None, cat='Continuous')

y = LpVariable.dicts('Plant_', [(i, c) for c in capacity for i in location], 
                      lowBound=None, upBound=None, cat='Binary')

## Define Constraints

In [8]:
# total production needs to be equal to the demand
for j in location:
    model += lpSum([X[(i, j)] for i in location]) == df_demand.loc[j, 'Demand']
    
# total production needs to be smaller or equal to total production capacity
for i in location:
    model += lpSum([X[i, j] for j in location]) <= lpSum([df_capacity.loc[i, c] * y[i, c] for c in capacity])

## Define Objective Function

In [9]:
model += (lpSum([df_fix_cost.loc[i, c] * y[(i, c)] for c in capacity for i in location]) + 
          lpSum([df_var_cost.loc[i, j] * X[(i, j)] for i in location for j in location]))

## Solve The Model!

In [10]:
print('The solution:', bool(model.solve()))
print('The status of solution:', LpStatus[model.status])
print('Results for production quantities:')

for elem in [{'Production':'{} to {}'.format(i,j), 'Quantity':X[(i,j)].varValue} for i in location for j in location]:
    print(f'* Quantity for production {elem["Production"]} is {elem["Quantity"]}')
          
print('Results for plant capacities based on location:')
for elem in [{'lowCap':y[(i, capacity[0])].varValue, 'highCap':y[(i, capacity[1])].varValue} for i in location]:
    print(f'* Low capacity: {elem["lowCap"]}, High capacity: {elem["highCap"]}')
          
print('The optimized objective function ($):', value(model.objective))

The solution: True
The status of solution: Optimal
Results for production quantities:
* Quantity for production A to A is 145.4
* Quantity for production A to B is 0.0
* Quantity for production A to C is 0.0
* Quantity for production A to D is 0.0
* Quantity for production A to E is 1219.6
* Quantity for production B to A is 0.0
* Quantity for production B to B is 0.0
* Quantity for production B to C is 0.0
* Quantity for production B to D is 0.0
* Quantity for production B to E is 0.0
* Quantity for production C to A is 0.0
* Quantity for production C to B is 84.1
* Quantity for production C to C is 156.4
* Quantity for production C to D is 176.8
* Quantity for production C to E is 0.0
* Quantity for production D to A is 0.0
* Quantity for production D to B is 0.0
* Quantity for production D to C is 0.0
* Quantity for production D to D is 1500.0
* Quantity for production D to E is 0.0
* Quantity for production E to A is 0.0
* Quantity for production E to B is 0.0
* Quantity for produc

---